# **Importing Libraries**

In [ ]:
pip install "dask[dataframe]" --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 141 kB 5.2 MB/s 


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc 
import warnings
import dask.dataframe as dd
from matplotlib.pyplot import figure
sns.set()
warnings.filterwarnings("ignore")

**To optimize memory consumption**

In [ ]:
def adjust_datatype(df):
    """
    This function adjust the datatypes of columns based upon their range
    
    Input take DataFrame
    
    example:
    if a column's maximum and minimum values are in range of (-128 to 127) then datatype
    to store that column is changed to int8
    
    Reference:
    https://www.kaggle.com/jeru666/did-you-think-of-these-features
    
    """
    int_cols = list(df.select_dtypes(include=['int']).columns)
    for col in int_cols:
        if ((np.max(df[col]) <= 127) and (np.min(df[col] >= -128))):
            df[col] = df[col].astype(np.int8)
        elif ((np.max(df[col]) <= 32767) and (np.min(df[col] >= -32768))):
            df[col] = df[col].astype(np.int16)
        elif ((np.max(df[col]) <= 2147483647) and (np.min(df[col] >= -2147483648))):
            df[col] = df[col].astype(np.int32)
        else:
            df[col] = df[col].astype(np.int64)          
    float_cols = list(df.select_dtypes(include=['float']).columns)
    for col in float_cols:
        df[col] = df[col].astype(np.float32)
        
        
        
        
        
def restart_kernel():
    """
    Used to get rid of unnessary variable
    
    https://stackoverflow.com/questions/37751120/restart-ipython-kernel-with-a-command-from-a-cell
    """
    from IPython.core.display import HTML
    HTML("<script>Jupyter.notebook.kernel.restart()</script>")
    import os
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    import gc 
    import warnings
    import dask.dataframe as dd
    from matplotlib.pyplot import figure
    sns.set()
    warnings.filterwarnings("ignore")

# **Data preprocessing and featuring engg of members_dataset**

In [ ]:
member_data=pd.read_csv('/content/drive/MyDrive/KKbox_data/members_v3.csv',parse_dates=['registration_init_time'])
member_data.head()

**After deriving conclusions from EDA, we have to fix some issues in the dataset**

In [ ]:
#Since, gender does not make any much difference, Lets drop it.
member_data.drop('gender',axis=1,inplace=True)

In [ ]:
#replace 0 instead of nan in registered_via
member_data['registered_via']=member_data['registered_via'].apply(lambda x:np.nan if x < 0 else x)
member_data['registered_via']=member_data.groupby('city')['registered_via'].transform(lambda x: x.fillna(x.value_counts().index[0]))
registered_via_mode=member_data['registered_via'].value_counts().index[0]

In [ ]:
# removing outliers
member_data['bd']=member_data['bd'].apply(lambda x :np.nan if (x <=10 or x >80) else x)
member_data['bd'].fillna(28.0, inplace = True)

In [ ]:
#From EDA, we know city 1 and 13 have more user churn, so 
member_data['city_feature']=member_data['city'].apply(lambda x:1 if (x==1 or x==13) else 0 )
member_data.drop('city',axis=1,inplace=True)

In [ ]:
#checks if user user registration method 7 and 9
member_data['registration_method_f'] = member_data['registered_via'].apply(lambda x :1 if (x==7 or x==9) else 0)

In [ ]:
member_data.to_csv('/content/drive/MyDrive/KKbox_data/New_preprocessed_Data/members_data_updated')

In [ ]:
restart_kernel()

# **Data preprocessing and feature engg of transaction data**

In [ ]:
transaction_data = pd.read_csv('/content/drive/MyDrive/KKbox_data/transactions_v2.csv',parse_dates=['transaction_date','membership_expire_date'])
transaction_data.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,2017-01-31,2017-05-04,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,2015-08-09,2019-04-12,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,2017-03-03,2017-04-22,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30,180,180,1,2017-03-29,2017-03-31,1
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30,99,99,1,2017-03-23,2017-04-23,0


In [ ]:
adjust_datatype(transaction_data)

In [ ]:
def preprocessing(data):
    #imputing 0 in place of nan value in payment_method_id
    data['payment_method_id'] = data['payment_method_id'].fillna(0)

    # removing outliers
    data['payment_plan_days'] = data['payment_plan_days'].apply(lambda x: x if (x <= 30.0) else np.nan)
    # imputing 30 in place of nan in payment_plan_days
    data['payment_plan_days'] = data['payment_plan_days'].fillna(30.0)

    # removing outliers
    data['plan_list_price'] = data['plan_list_price'].apply(lambda x: x if (x <= 180.0) else np.nan)
    # imputing 149 in place of nan in plan_list_price
    data['plan_list_price'] = data['plan_list_price'].fillna(149.0)

    # removing outliers
    data['actual_amount_paid'] = data['actual_amount_paid'].apply(lambda x: x if (x <= 180.0) else np.nan)
    # imputing 149 in place of nan in actual_amount_paid
    data['actual_amount_paid'] = data['actual_amount_paid'].fillna(149.0)

    # imputing 2 in place of nan values in is_auto_renew
    data['is_auto_renew'] = data['is_auto_renew'].fillna(2)

    # filling median date in place of nan in the df
    data['transaction_date'] = data['transaction_date'].fillna(20170316.0)
    # converting float date to datetime
    data['transaction_date'] = pd.to_datetime(data['transaction_date'], format='%Y%m%d')

    # filling median date in place of nan in the df
    data['membership_expire_date'] = data['membership_expire_date'].fillna(20170419.0)
    # converting float date to datetime
    data['membership_expire_date'] = pd.to_datetime(data['membership_expire_date'], format='%Y%m%d')

    # imputing 2 in place of nan values in is_cancel
    data['is_cancel'] = data['is_cancel'].fillna(2)

    return data

In [ ]:
preprocessing(transaction_data)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,30.0,149.0,149.0,0,2017-01-31,2017-05-04,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30.0,149.0,149.0,1,2015-08-09,2019-04-12,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30.0,180.0,180.0,1,2017-03-03,2017-04-22,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30.0,180.0,180.0,1,2017-03-29,2017-03-31,1
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30.0,99.0,99.0,1,2017-03-23,2017-04-23,0
...,...,...,...,...,...,...,...,...,...
1431004,zwF50wwaJI2TBKWhB42HRBJ6EQK0jgSo1Xmwb9Jq3SU=,32,30.0,149.0,149.0,0,2017-02-15,2017-08-17,0
1431005,zx/h5MzQQmsSat04wSfGpHp6N8aWLLwM1+7OV7ujmPY=,41,30.0,149.0,149.0,1,2017-03-06,2017-04-06,0
1431006,zxvgjIKjy18Fm+cIWUfYKr68z09+ILBxuMW0DnbeUZ8=,41,30.0,99.0,99.0,1,2017-03-08,2017-04-08,0
1431007,zzNhkExbpzmpjp9tXefiCUBtgNLgS+vZE7fFfTRDJVc=,38,30.0,149.0,149.0,0,2017-03-18,2017-04-17,0


**Adding some more features to the transaction dataset**

In [ ]:
#Creating duration feature
temp=transaction_data['membership_expire_date']-transaction_data['transaction_date']
transaction_data['duration']=temp.apply(lambda x :x.days)

In [ ]:
temp_df = transaction_data.groupby('msno').agg( is_auto_renew_change_feature = ('is_auto_renew', 'max'),
                                               is_cancel_change_feature = ('is_cancel','max'),
                                               average_plan = ('payment_plan_days','mean'),
                                               average_amount_paid = ('actual_amount_paid','mean'),
                                               average_amount_charged = ('plan_list_price','mean'),
                                               change_in_plan = ('payment_plan_days','nunique'))



In [ ]:
transaction_data = pd.merge(transaction_data, temp_df, on='msno', how='left')

In [ ]:
transaction_data = transaction_data.drop_duplicates('msno', keep='first', inplace=False)

In [ ]:
adjust_datatype(transaction_data)

In [ ]:
transaction_data.to_csv('/content/drive/MyDrive/KKbox_data/New_preprocessed_Data/transaction_data_updated')

# **Data preprocessing and feature engg of user_logs data**

In [ ]:
user_logs=pd.read_csv("/content/drive/MyDrive/KKbox_data/user_logs_v2.csv",parse_dates=['date'])
user_logs.head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=,2017-03-31,8,4,0,1,21,18,6309.273
1,nTeWW/eOZA/UHKdD5L7DEqKKFTjaAj3ALLPoAWsU8n0=,2017-03-30,2,2,1,0,9,11,2390.699
2,2UqkWXwZbIjs03dHLU9KHJNNEvEkZVzm69f3jCS+uLI=,2017-03-31,52,3,5,3,84,110,23203.337
3,ycwLc+m2O0a85jSLALtr941AaZt9ai8Qwlg9n0Nql5U=,2017-03-31,176,4,2,2,19,191,7100.454
4,EGcbTofOSOkMmQyN1NMLxHEXJ1yV3t/JdhGwQ9wXjnI=,2017-03-31,2,1,0,1,112,93,28401.558


In [ ]:
user_logs.isnull().values.any()

False

In [ ]:
user_logs.duplicated().values.any()

False

In [ ]:
adjust_datatype(user_logs)

In [ ]:
def preprocessing_for_userlogs(data):
   # filling median date in place of nan in the df
    data['date'] = data['date'].fillna(20170316.0)
    # converting float date to datetime
    data['date'] = pd.to_datetime(data['date'], format='%Y%m%d')

    # removing outliers, impute median value is 2
    data['num_25'] = data['num_25'].apply(lambda x: x if (x <= 15.0) else np.nan)
    # now I can impute 2 instead of nan in num_25
    data['num_25'] = data['num_25'].fillna(2.0)

    # removing outliers, impute median value 0
    data['num_50'] = data['num_50'].apply(lambda x: x if (x <= 4.0) else np.nan)
    # now I can impute 0 instead of nan in num_50
    data['num_50'] = data['num_50'].fillna(0)

    # removing outliers, impute median value 0
    data['num_75'] = data['num_75'].apply(lambda x: x if (x <= 3.0) else np.nan)
    # now I can impute 0 instead of nan in num_75
    data['num_75'] = data['num_75'].fillna(0)

    # removing outliers, impute median value 0
    data['num_985'] = data['num_985'].apply(lambda x: x if (x <= 3.0) else np.nan)
    # now I can impute 0 instead of nan in num_985
    data['num_985'] = data['num_985'].fillna(0)

    # removing outliers, impute median value 14
    data['num_100'] = data['num_100'].apply(lambda x: x if (x <= 74.0) else np.nan)
    # now I can impute 14 instead of nan in num_100
    data['num_100'] = data['num_100'].fillna(14.0)

    # removing outliers, impute median value 16
    data['num_unq'] = data['num_unq'].apply(lambda x: x if (x <= 68.0) else np.nan)
    # now I can impute 16 instead of nan in num_unq
    data['num_unq'] = data['num_unq'].fillna(16.0)

    # removing outliers, impute median value 3880
    data['total_secs'] = data['total_secs'].apply(lambda x: x if (x <= 19167.549700000025) else np.nan)
    # now I can impute 3880.765 instead of nan in total_secs
    data['total_secs'] = data['total_secs'].fillna(3880.765)
    
    return data

In [ ]:
preprocessing_for_userlogs(user_logs)

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=,2017-03-31,8.0,4.0,0.0,1.0,21.0,18.0,6309.272949
1,nTeWW/eOZA/UHKdD5L7DEqKKFTjaAj3ALLPoAWsU8n0=,2017-03-30,2.0,2.0,1.0,0.0,9.0,11.0,2390.698975
2,2UqkWXwZbIjs03dHLU9KHJNNEvEkZVzm69f3jCS+uLI=,2017-03-31,2.0,3.0,0.0,3.0,14.0,16.0,3880.765000
3,ycwLc+m2O0a85jSLALtr941AaZt9ai8Qwlg9n0Nql5U=,2017-03-31,2.0,4.0,2.0,2.0,19.0,16.0,7100.454102
4,EGcbTofOSOkMmQyN1NMLxHEXJ1yV3t/JdhGwQ9wXjnI=,2017-03-31,2.0,1.0,0.0,1.0,14.0,16.0,3880.765000
...,...,...,...,...,...,...,...,...,...
18396357,FGpiy2mB+vXLKziYRcY/xJcJEFJfRDfUqlU+p760f7E=,2017-03-14,0.0,0.0,0.0,0.0,1.0,1.0,248.057999
18396358,iZRjKNMrw5ffEbfXODLhV/0tJLPbOH3am1WYDgqBf8Q=,2017-03-06,0.0,0.0,0.0,0.0,1.0,1.0,311.000000
18396359,yztw4Y0EggG0w2wPkbMZx7ke7saSx7dLSfMheHZG/DQ=,2017-03-31,0.0,0.0,0.0,0.0,17.0,1.0,3973.188965
18396360,swCHwkNx30/aENjq30qqaLlm7bUUytbMXdz1bH7g0Jk=,2017-03-07,0.0,0.0,0.0,1.0,0.0,1.0,179.278000


In [ ]:
adjust_datatype(user_logs)

In [ ]:
user_logs = user_logs.drop_duplicates('msno', keep='first', inplace=False)

In [ ]:
user_logs.shape

(1103894, 9)

**Adding some more features to user logs data**

In [ ]:
temp_df1 = user_logs.groupby('msno').agg(total_secs_mean = ('total_secs', 'mean'))

In [ ]:
user_logs = pd.merge(user_logs, temp_df1, on='msno', how='left')

In [ ]:
user_logs = user_logs.drop_duplicates('msno', keep='first', inplace=False)

In [ ]:
adjust_datatype(user_logs)

In [ ]:
user_logs.to_csv('/content/drive/MyDrive/KKbox_data/New_preprocessed_Data/user_logs_updated')